# Obteniendo los datos estadisticos de todos los archivos

Se calculan todas las variables estadisticas y se almacenan en un dataframe

### Importando librerias

In [21]:
import pandas as pd
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import math
import numpy as np
import matplotlib.pyplot as plt

### Importar lista con nombres de archivos

*Cambiar el nombre de la lista según el sentimiento a analizar

In [22]:
files = pd.read_csv("Sadness.csv") ### MODIFICAR EL NOMBRE

### Se crea un diccionario para almacenar las variables de cada archivo

In [23]:
data_instance = {'M1': [],
                 'M2': [],
                 'M3': [],
                 'M4': [],
                 'C2': [],
                 'C3': [],
                 'C4': [],
                 'Skewness': [],
                 'Kurtosis': [],
                 'Mean': [],
                 'Absolute_mean': [],
                 'Peak_value': [],
                 'SRV': [],
                 'RMS': [],
                 'Crest_factor': [],
                 'Shape_factor': []}

### Ciclo para leer todos los archivos de cada sentimiento

In [24]:
for a in range(int(files.count())):
    name_doc = files._get_value (a, 'Name')
    data = pd.read_csv (name_doc)



    ########################################################################################################################################
    ################# Fourier #################
    raw_data = np.ravel(data.to_numpy())
    n = len(raw_data)
    yf = np.fft.fft(raw_data)       #Get fourier transform
    yfts = np.fft.fftshift(yf)
    yftsa = 1.0/n * np.abs(yfts)        #Remove complex numbers

    # Convert numpy array to DataFrame
    data = pd.DataFrame (yftsa)
    ##########################################################################################################################################


    ################# Get statistical features #################
    m1 = float (stats.moment (data, moment = 1)) #First moment 
    m2 = float (stats.moment (data, moment = 2)) #Second moment
    m3 = float (stats.moment (data, moment = 3))#Third moment
    m4 = float (stats.moment (data, moment = 4)) #Fourth moment
    sc = float (m2 - math.sqrt (pow (m1,2))) #Second cumulant
    tc = float (m3 - (3*m1*m2) - (2*pow (m1,3))) #Third cumulant
    fc = float (m4 + (3*m3*m1) - (3*pow (m2,2)) + (12*m2*pow (m1,2)) - (6*pow (m1,4))) #Fourth cumulant
    skew = float (stats.skew (data)) #Skewness
    kurt = float (stats.kurtosis (data)) #Kurtosis
    mean = float(data.mean()) #Mean
    am = abs(float(data.mean())) #Absolute mean
    pv = 0.5*(float(data.max()) - float(data.min())) #Peak value
    da = abs(data)
    dsa = da**2
    srv = float(pow((dsa.sum()/int(dsa.count())),2)) #Square root value
    d2 = data**2
    rms = math.sqrt (float(d2.sum())/int(d2.count())) #Root mean square
    cf = pv/rms #Crest factor
    sf = rms/am #Shape factor

    ################# Append features to a dictionary #################
    data_instance ['M1'].append(m1)
    data_instance ['M2'].append(m2)
    data_instance ['M3'].append(m3)
    data_instance ['M4'].append(m4)
    data_instance ['C2'].append(sc)
    data_instance ['C3'].append(tc)
    data_instance ['C4'].append(fc)
    data_instance ['Skewness'].append(skew)
    data_instance ['Kurtosis'].append(kurt)
    data_instance ['Mean'].append(mean)
    data_instance ['Absolute_mean'].append(am)
    data_instance ['Peak_value'].append(pv)
    data_instance ['SRV'].append(srv)
    data_instance ['RMS' ].append(rms)
    data_instance ['Crest_factor'].append(cf)
    data_instance ['Shape_factor'].append(sf)

    del data

### Se crea un dataframe con todos los datos estadisticos y se guarda en un .csv

In [25]:
################# Create a dataframe from dictionary #################
data_stats = pd.DataFrame(data=data_instance)
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,4.637135e-08,5.113475e-10,1.683434e-11,4.637135e-08,5.113475e-10,1.682789e-11,51.208434,7825.825986,0.000026,0.000026,0.021008,2.211372e-15,0.000217,96.875990,8.480430
1,0.0,4.905584e-08,4.633225e-10,1.419202e-11,4.905584e-08,4.633225e-10,1.418480e-11,42.642955,5894.428256,0.000031,0.000031,0.019667,2.504718e-15,0.000224,87.910934,7.105317
2,0.0,7.338463e-08,7.723601e-10,2.321654e-11,7.338463e-08,7.723601e-10,2.320039e-11,38.851894,4308.092344,0.000033,0.000033,0.019666,5.543460e-15,0.000273,72.074388,8.342509
3,0.0,3.416386e-08,2.945406e-10,9.289916e-12,3.416386e-08,2.945406e-10,9.286414e-12,46.643892,7956.355283,0.000027,0.000027,0.019697,1.216804e-15,0.000187,105.464139,6.966010
4,0.0,5.488842e-08,4.251065e-10,1.232066e-11,5.488842e-08,4.251065e-10,1.231162e-11,33.058010,4086.521710,0.000035,0.000035,0.019864,3.149625e-15,0.000237,83.850165,6.746305


In [26]:
#Normalizacion
scaler = MinMaxScaler()
data_stats = pd.DataFrame (scaler.fit_transform(data_stats),columns=['M1','M2','M3','M4','C2','C3','C4','Skewness','Kurtosis','Mean','Absolute_mean','Peak_value','SRV','RMS','Crest_factor','Shape_factor'])

data_stats.to_csv (r"..\stats\Normalizated_data\Stats_sadness_fourier.csv")
data_stats.head()

,M1,M2,M3,M4,C2,C3,C4,Skewness,Kurtosis,Mean,Absolute_mean,Peak_value,SRV,RMS,Crest_factor,Shape_factor
0,0.0,0.236891,0.500407,0.703663,0.236891,0.500407,0.703470,0.376565,0.282351,0.258885,0.258885,0.957481,0.053114,0.466520,0.608175,0.891467
1,0.0,0.250642,0.453407,0.593216,0.250642,0.453407,0.592979,0.300999,0.211766,0.323606,0.323606,0.894280,0.060160,0.482101,0.535649,0.591191
2,0.0,0.375269,0.755849,0.970435,0.375269,0.755849,0.969865,0.267554,0.153792,0.336984,0.336984,0.894266,0.133147,0.593747,0.407534,0.861350
3,0.0,0.174357,0.288227,0.388311,0.174357,0.288227,0.388207,0.336296,0.287121,0.272460,0.272460,0.895727,0.029226,0.398185,0.677652,0.560771
4,0.0,0.280520,0.416007,0.514994,0.280520,0.416007,0.514673,0.216439,0.145695,0.363335,0.363335,0.903580,0.075650,0.512056,0.502798,0.512795
